In [114]:
#Librerias para trabajar con la red Bayesiana
import pandas as pd
from pgmpy.models import BayesianModel
from pgmpy.estimators import MaximumLikelihoodEstimator
from pgmpy.inference import VariableElimination

In [115]:
# Cargar los datos desde el archivo CSV
ArchivoCSV = "../tablaOriginalVersion2.csv"
#ArchivoCSV = "../tabla_gruposNewNoSeason.csv"
data = pd.read_csv(ArchivoCSV)

# Visualizar los datos cargados
print(data.head())

   semestre  grupos_1  grupos_2  grupos_3  grupos_4  grupos_5  grupos_6  \
0         1       0.0         2         3         1         4         2   
1         2      15.0        24        19        19        19        32   
2         3      18.0        19        22        18        18        18   
3         4      12.0        13        17        18        18        18   
4         5      18.0        14        12        12        12        17   

   grupos_7  grupos_8  grupos_9  ...  grupos_13  grupos_14  grupos_15  \
0         2         1         2  ...          3          4          3   
1        14        26        22  ...         20         22         21   
2        24        14        13  ...         21         18         19   
3        17        23        22  ...         16         18         18   
4        18        16        18  ...         18         12         17   

   grupos_16  grupos_17  grupos_18  grupos_19  grupos_20  grupos_21  grupos_22  
0          4          3      

In [43]:
data.head()

,semestre,grupos_1,grupos_2,grupos_3,grupos_4,grupos_5,grupos_6,grupos_7,grupos_8,grupos_9,...,grupos_13,grupos_14,grupos_15,grupos_16,grupos_17,grupos_18,grupos_19,grupos_20,grupos_21,grupos_22
0,1,0.0,2,3,1,4,2,2,1,2,...,3,4,3,4,3,3,3,6,3,3
1,2,15.0,24,19,19,19,32,14,26,22,...,20,22,21,22,24,30,23,26,21,23
2,3,18.0,19,22,18,18,18,24,14,13,...,21,18,19,18,20,19,24,23,24,20
3,4,12.0,13,17,18,18,18,17,23,22,...,16,18,18,18,19,20,18,24,18,18
4,5,18.0,14,12,12,12,17,18,16,18,...,18,12,17,12,18,18,18,18,17,14


In [116]:
# Crear el modelo de red bayesiana
modelo_bayesiano = BayesianModel()


c:\Users\ernes\AppData\Local\Programs\Python\Python311\Lib\site-packages\pgmpy\models\BayesianModel.py:8: FutureWarning: BayesianModel has been renamed to BayesianNetwork. Please use BayesianNetwork class, BayesianModel will be removed in future.
  warnings.warn(


In [117]:
# Crear los nodos del modelo
for i in range(1, 11):
    modelo_bayesiano.add_node(f'grupos_{i}')

# Definir las relaciones entre las variables (aristas de la red bayesiana)
for i in range(1, 10):
    modelo_bayesiano.add_edge(f'grupos_{i}', f'grupos_{i+1}')

In [118]:
# Agregar el nodo 'grupos_10_prediccion' al modelo
modelo_bayesiano.add_node('grupos_10_prediccion')

In [119]:
# Entrenar el modelo usando MaximumLikelihoodEstimator
entrenador = MaximumLikelihoodEstimator(modelo_bayesiano, data)


ValueError: variable names of the model must be identical to column names in data

In [107]:
#Imprimir cantidad de nodos en el modelo
print(modelo_bayesiano.nodes())


['grupos_1', 'grupos_2', 'grupos_3', 'grupos_4', 'grupos_5', 'grupos_6', 'grupos_7', 'grupos_8', 'grupos_9', 'grupos_10']


In [108]:
# Estimar las probabilidades basadas en los datos
cpds = {}
for i in range(1, 11):
    cpds[f'grupos_{i}'] = entrenador.estimate_cpd(f'grupos_{i}')

# Agregar CPDs a la red bayesiana
for i in range(1, 11):
    modelo_bayesiano.add_cpds(cpds[f'grupos_{i}'])

In [109]:
# Realizar la inferencia para obtener las predicciones
inference = VariableElimination(modelo_bayesiano)

In [110]:
# Realizar la predicción para cada semestre
predicciones = []
for semestre in range(1, data['semestre'].max() + 1):
    evidencia = {}
    for i in range(1, 11):
        grupo = data.loc[data['semestre'] == semestre, f'grupos_{i}'].values.astype(int)[0]
        evidencia[f'grupos_{i}'] = grupo
    resultado = inference.map_query(variables=['grupos_10_prediccion'], evidence=evidencia)  # Cambiamos el nombre de 'grupos_10' a 'grupos_10_prediccion'
    predicciones.append((semestre, resultado['grupos_10_prediccion'][0]))  # Tomamos el valor escalar de la predicción




NetworkXError: The node grupos_10_prediccion is not in the digraph.

In [59]:
# Mostrar las predicciones
for pred in predicciones:
    print(f"Semestre: {pred[0]}, Número de grupos predicho: {pred[1]}")

In [60]:
predicciones

[]